## Imports and Logging

In [1]:
import os
import pandas as pd
import logging as log
import datetime as dt
from textwrap import TextWrapper as txwr

if not os.path.isdir ("Logs") :
    os.mkdir ("Logs")
     
def dateFmt () :
    """Date component of the run log file"""
    dtStr = str(dt.datetime.now()) 
    dtStr = dtStr[:dtStr.find('.')]
    dtStr = dtStr.replace(' ', '_')
    return dtStr

runlog = log.getLogger (__name__)
runlog.setLevel (log.DEBUG)
runLogPath = "Logs/run_{}.log".format(dateFmt())
fileHandler = log.FileHandler (runLogPath)
runlog.addHandler (fileHandler)
runlog.info("Batch runner started!")

In [2]:
def logFixFmt (fix, k=50) :
    """Formats error messages for the run logger"""
    return  2*(k*"#" + '\n') + txwr(width=k).fill(text=fix) + '\n' + 2*(k*"#" + '\n')

def runPrelude (batchName, csv) :
    """Sets up files and folders and checks for existence"""

    global fileHandler
    fileHandler.setFormatter (log.Formatter ("%(levelname)s : RUN_INIT : %(asctime)s : %(message)s", 
                         datefmt='%m/%d/%Y %I:%M:%S %p'))

    runlog.info("Entering prelude to create new batch!")
    if not os.path.isdir("Data") :
        runlog.error("Data folder not found!\n\n{}".format(logFixFmt(
            "Please create a folder named 'Data' in the notebook directory and rerun!"
        )))
        raise FileNotFoundError

    batchPath = "Data/{}".format(batchName)
    if not os.path.isdir (batchPath) :
        runlog.error("Batch folder not found\n\n{}".format(logFixFmt(
            "Please create a folder for the batch at '{}' and rerun!".format(batchPath)
        )))
        raise FileNotFoundError

    csvName = batchName if csv is None else csv
    csvPath = batchPath + "/{}".format(csvName + ".csv")
    if not os.path.exists (csvPath) :
        runlog.error("Batch .csv file at path '{}' not found\n\n{}".format(batchPath, logFixFmt(
            "Please supply the name of the appropriate .csv file and rerun!"
        )))
        raise FileNotFoundError

    runlog.info("Changing log format to handle batch '{}'".format(batchName))
    fmtBatch = log.Formatter ("%(levelname)s : {} : %(asctime)s : %(message)s".format(batchName), 
                     datefmt='%m/%d/%Y %I:%M:%S %p')
    fileHandler.setFormatter (fmtBatch)

    fitsPath = os.path.join (batchPath, "FITS")
    if not os.path.exists (fitsPath) :
        os.mkdir (fitsPath)
        runlog.info("Created FITS folder for batch")
    else :
        runlog.info("FITS folder for the batch already exists")
        
    if (batchlog:=Batch.logFHs[batchName]) is None :
        batchlog = log.getLogger (batchName) 
        batchlog.setLevel (log.INFO)
        batchHandler = log.FileHandler (os.path.join(batchPath, csvName+".log"))
        batchHandler.setFormatter (log.Formatter ("%(levelname)s : {} : %(asctime)s : %(message)s".format(batchName), 
                                datefmt='%m/%d/%Y %I:%M:%S %p'))
        batchlog.addHandler (batchHandler)
        Batch.logFHs[batchName] = batchlog
        
        runlog.info("Added batch logger for this run")
        batchlog.info ("Batch object created with fresh logger")
    else :
        batchlog.info("Batch object created with existing logger")
        
    runlog.info("Successfully created environment for batch\n\nMonitoring...")
    return csvPath

## Batch Class

In [3]:
class Batch () :
    """Class that defines a batch of SDSS objIDs on which classifcation is to be performed"""
    
    logFHs = {}
    
    def getBatch (batchName, csv=None) :
        """Class method to get a batch"""
        try :
            batch = Batch (batchName, batchName if csv is None else csv)
        except FileNotFoundError :
            print ("Error initialising batch!")
            print("Kindly check the latest message in the logfile '{}' for a fix.".format(
                os.path.join(os.getcwd(), runLogPath)
            ))
            print ("Abort!")
            batch = None
        finally :
            return batch
    
    def __init__ (self, batchName, csv=None) :
        """Initialises the batch"""
        
        if batchName not in Batch.logFHs :
            Batch.logFHs[batchName] = None
    
        self.csvPath = runPrelude (batchName, csv)
        self.log = Batch.logFHs[batchName]
        self.idList = (lambda col:pd.read_csv(self.csvPath, usecols=[col])[col])("objID")
        
        self.galaxies = {}
        for objid in self.idList :
            self.galaxies[objid] = None
            
    def downloadPhase (self) :
        runlog.info ("Downloading currently monitoring batch")
        for objid in self.idList :
            self.log.info ("Downloaded {}".format(objid))
        self.log.info ("Downloaded batch")
        runlog.info ("Downloaded currently monitoring batch")
        
    @property
    def csvName (self) :
        """Plain name of the csv File"""
        return (lambda st:st[1+st.rfind('/'):st.find(".csv")])(self.csvPath)

In [4]:
batch = Batch.getBatch ("Debug")

In [6]:
batch.downloadPhase ()